<a href="https://colab.research.google.com/github/ucfilho/marquesgabi/blob/master/Calculo_014_out_30_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# carrega as bibliotecas do python

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import os 
import cv2
import zipfile
from skimage import data
from skimage import filters
from skimage import exposure
import skimage.feature as sk
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.metrics.cluster import entropy
from sklearn.metrics.cluster import homogeneity_score


In [3]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip


--2019-10-30 14:50:04--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip.5’

Sugar_set_30_2019.z 100%[===================>]  18.68M  3.90MB/s    in 4.8s    

2019-10-30 14:50:10 (3.90 MB/s) - ‘Sugar_set_30_2019.zip.5’ saved [19584173/19584173]



In [0]:

file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall() 

In [0]:
# all images are resized in variable x, are in gray in variable p and in variable w are dataframe
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=5 # trocar por 64 qdo acabar a analise

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol))
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
Nfotos=len(img_name)
GLCM=np.zeros((Nfotos, 14))   # lembrar do ANN_Hydrofoil_Neuronio_set_17_2018.ipynb
Nrow=Size
Ncol=Size
#print(GLCM.shape)
#Soma=np.sum(pp[k,:])
for k in range(Nfotos):
  
  p=pp[k,:].reshape(Size,Size)
  Soma=0
  for i in range(Size):
    for j in range(Size):
      Soma=Soma+p[i,j]
   
  for i in range(Size):
    for j in range(Size):
      p[i,j]=p[i,j]/Soma
  
  G=Ncol-1
  Pdif=[]
  Soma_dif=0
  Psom=[]
  Soma_soma=0

  for kk in range(2*G+1):
    for i in range(Nrow):
      for j in range(Ncol):
        if((i+j)==kk):
          Soma_soma=Soma_soma+p[i,j]
        if((abs(i-j))==kk):
          Soma_dif=Soma_dif+p[i,j]
    Psom.append(Soma_soma)

  for kk in range(G+1):
    for i in range(Nrow):
      for j in range(Ncol):
        if((abs(i-j))==kk):
          Soma_dif=Soma_dif+p[i,j]
    Pdif.append(Soma_dif)

  for i in range(Nrow):
    for j in range(Ncol):
      #print("i=%d j=%d k=%d"%(i,j,k))
      GLCM[k,0]=GLCM[k,0]+p[i,j]**2 #segundo momento angular
      GLCM[k,1]=GLCM[k,1]+(i-j)**2*p[i,j] #contraste


      MIx=0;MIy=0
      for ii in range(Nrow):
        for jj in range(Ncol):
          MIy=MIy+jj*p[ii,jj]
          MIx=MIx+ii*p[ii,jj]
      Sx2=0;Sy2=0;
      for ii in range(Nrow):
        for jj in range(Ncol):
          Sx2=Sx2+(ii-MIx)**2*p[ii,jj]
          Sy2=Sy2+(jj-MIy)**2*p[ii,jj]
      Sx=Sx2**0.5
      Sy=Sy2**0.5


      GLCM[k,2]=GLCM[k,2]+((i*j)*p[i,j]-MIx*MIy)/(Sx*Sy) #Correlacao
      # CONFERIR CORRELACAO EM OUTROS

      GLCM[k,3]=GLCM[k,3]+(i-np.mean(p))**2*p[i,j] #Variancia

      GLCM[k,4]=GLCM[k,4]+(1/(1+(i-j)**2))*p[i,j]#INVERSA DIF MOMENTO

      #GLCM[k,5]=GLCM[k,5]+(1/(1+(i-j)**2))*p[i,j]#INVERSA DIF MOMENTO

      GLCM[k,8]=GLCM[k,8]-p[i,j]*np.log(p[i,j]) # entropia



In [0]:
# ver os conceitos aqui tambem
# https://www.uio.no/studier/emner/matnat/ifi/INF4300/h08/undervisningsmateriale/glcm.pdf

In [8]:
pd.set_option('display.max_columns', 30)
print(pd.DataFrame(GLCM))

           0         1          2         3         4    5    6    7  \
0   0.040159  4.006764 -48.388893  5.873156  0.423837  0.0  0.0  0.0   
1   0.041540  4.282528 -47.724904  5.762803  0.411868  0.0  0.0  0.0   
2   0.040287  4.030399 -49.193342  5.979967  0.424910  0.0  0.0  0.0   
3   0.040175  4.015663 -48.607804  6.027608  0.435603  0.0  0.0  0.0   
4   0.040313  4.043838 -47.746904  5.656749  0.422210  0.0  0.0  0.0   
5   0.040234  4.056964 -49.752577  5.948915  0.431240  0.0  0.0  0.0   
6   0.040177  3.895373 -49.375098  5.944445  0.435292  0.0  0.0  0.0   
7   0.040276  3.931919 -51.702480  6.036687  0.428104  0.0  0.0  0.0   
8   0.040229  3.993338 -49.941784  6.066993  0.427480  0.0  0.0  0.0   
9   0.040272  3.979461 -49.007498  5.888981  0.430643  0.0  0.0  0.0   
10  0.040134  3.934069 -48.671268  5.729726  0.432728  0.0  0.0  0.0   
11  0.041463  3.928237 -48.569188  5.890711  0.447866  0.0  0.0  0.0   
12  0.040367  4.035181 -49.472654  5.853991  0.426656  0.0  0.0 

In [9]:
GLCM=pd.DataFrame(GLCM)
pd.set_option('display.max_columns', 30)
print(GLCM.head(5))

          0         1          2         3         4    5    6    7         8  \
0  0.040159  4.006764 -48.388893  5.873156  0.423837  0.0  0.0  0.0  3.216867   
1  0.041540  4.282528 -47.724904  5.762803  0.411868  0.0  0.0  0.0  3.203164   
2  0.040287  4.030399 -49.193342  5.979967  0.424910  0.0  0.0  0.0  3.215219   
3  0.040175  4.015663 -48.607804  6.027608  0.435603  0.0  0.0  0.0  3.216685   
4  0.040313  4.043838 -47.746904  5.656749  0.422210  0.0  0.0  0.0  3.214975   

     9   10   11   12   13  
0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  
